# PI7 Onderzoeksrapport
#### Dajmen Graus en Bram Verheijen 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
import matplotlib.pyplot as plt



df = pd.read_csv('winequality-red.csv')


print(df.columns)

## Data Selectie

## Uitwerkingen

In [ ]:
y = df['quality']
X = df.drop('quality',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

model = LinearRegression()

model.fit(X_train, y_train)

predictions = model.predict(X_test)

print(
  'mean_squared_error : ', mean_squared_error(y_test, predictions))
print(
  'mean_absolute_error : ', mean_absolute_error(y_test, predictions))


xfit = np.linspace(0,100)
plt.plot(X_train, predictions)

## Conclusie